Neural Model

In [1]:
import random
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import torch.cuda
import numpy as np

# inspect torch path file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("deam_processed/best_model.pth", map_location=device)
print(checkpoint.keys())
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")
torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'output.weight', 'output.bias'])
CUDA available: False
CUDA version: None
GPU count: 0
Using cpu device


Load data

In [21]:
import pandas as pd

# grab MERGE data
# VGGish
verbose = False
nn_features = np.load("data/Audio Features and Model/extracted_features/extracted_features/vggish_features_meanpool.npz")
if True:
    print('nn_features')
    print(nn_features.files)
    print(nn_features["filenames"])
    print(nn_features["features"].shape)

#X_audio
ids_train = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_train.npy")
if verbose:
    print('ids_train')
    print(ids_train.shape)
    print(ids_train)

ids_val = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_val.npy")
if verbose:
    print('ids_val')
    print(ids_val.shape)
    print(ids_val)

ids_test = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_test.npy")
if verbose:
    print('ids_test')
    print(ids_test.shape)
    print(ids_test)

# RoBERTa
# X_lyrics
panda_merge_lyrics_features = pd.read_csv("data/Lyrics Features and Model/lyrics_embeddings_FULL/embeddings_128dim_with_metadata.csv")
if False:
    print(panda_merge_lyrics_features.columns)
    print(panda_merge_lyrics_features.head())

test_lyrics_features = np.load("data/Lyrics Features and Model/lyrics_embeddings_FULL/test_embeddings_128dim.npy")
if True:
    print('test_lyrics_features')
    print(test_lyrics_features.shape)
    # print(test_lyrics_features)

# Y
Y_pred = pd.read_csv("data/MERGE_Bimodal_Complete/merge_bimodal_complete_av_values.csv")
if verbose:
    print(Y_pred.columns)
    print(Y_pred.head())


nn_features
['features', 'labels', 'filenames']
['A001' 'A002' 'A003' ... 'MT0036368550' 'MT0039250520' 'MT0041030749']
(2216, 128)
test_lyrics_features
(384, 128)


Preprocessing

In [47]:
def extract_features(audio_ids, audio_features_file, lyrics_features, truth_pred):
    """
    Extract V/A features from three data sets representing the audio, lyrics and true predictions of VA values.
    :param audio_ids: file of ids to identity songs in split
    :param audio_features: audio features
    :param lyrics_features: lyrics features
    :param truth_pred: A/V true features
    :return: np.arrays X, Y, shape N,4 and N,2 respectively
    """
    X = np.empty((1, 128))
    Y = np.empty((1, 2))
    count = 0
    audio_feature_name = np.array(audio_features_file['filenames'])
    audio_features = np.array(audio_features_file['features'])
    print(audio_features[0].shape)
    # print(audio_ids)
    missing_count = 0
    for i in range(audio_ids.shape[0]):
        song_id = audio_ids[i] # get song id in training splits
        Y_row = truth_pred[truth_pred["Audio_Song"] == song_id]
        lyrics_id = Y_row["Lyric_Song"]
        if len(lyrics_id) == 0:
            print("song_id: ", song_id, ", missing in lyrics_id")
            continue
        if False:
            print(lyrics_id)
        feature_loc = np.where(audio_feature_name == song_id)[0]
        lyrics_id = lyrics_id.item()
        lyrics_features_row = lyrics_features[lyrics_features['song_id'] == lyrics_id]
        # print(lyrics_features_row.values)
        if (lyrics_features_row.shape[0] == 0):
            missing_count += 1
            if verbose:
                print("song_id: ", song_id, ", missing lyrics features")
            continue
        lyrics_feature = lyrics_features_row.values[0][1:129]
        audio_feature = np.array(audio_features[feature_loc]).flatten()
        y_bimodal = np.flip(Y_row.values[0][2:4])
        # print(np.sum(lyrics_feature))
        xEntry = np.concatenate([lyrics_feature])
        X = np.vstack([X, xEntry])
        Y = np.vstack([Y, y_bimodal])
        count+=1
    print('run stats')
    print('count: ', count)
    print('missing: ', missing_count)
    print('X shape: ', X.shape)
    print('Y shape: ', Y.shape)
    np.savetxt('features.txt', X,
           fmt='%.6f',           # Format (2 decimal places)
           delimiter=',')
    return X, Y

X_train, Y_train = extract_features(audio_ids=ids_train, audio_features_file=nn_features, lyrics_features=panda_merge_lyrics_features, truth_pred=Y_pred)

(128,)
run stats
count:  258
missing:  1293
X shape:  (259, 128)
Y shape:  (259, 2)


NN modelling